## Ejemplos del HMM Like

### Nombre

* baseline_HMMLike_zero_50_0_0_k_10.crfsuite

### Parametros

* l1 = 0
* l2 = 0
* Precisión = 0.8762
* Max_Iter = 50

In [1]:
import os
import pycrfsuite
import sys  
import random
sys.path.insert(0, '../')
from utils import get_corpus, WordsToLetter, extractLabels, extractTokens

model = "baseline_HMMLike_zero_50_0_0_k_10.crfsuite"

In [2]:
model_path = os.path.join("../models/baseline", "tsu_" + model)
print(model_path)
tagger = pycrfsuite.Tagger()
# Cargando modelos preentrenados
tagger.open(model_path)

../models/baseline/tsu_baseline_HMMLike_zero_50_0_0_k_10.crfsuite


In [3]:
# Obteniendo corpus para pruebas
corpus = get_corpus('corpus_otomi_mod', '../corpora/') + get_corpus('corpus_hard', '../corpora/')
letter_corpus = WordsToLetter(corpus)

In [4]:
# funciones auxiliares
def obtener_palabras(frases):
    palabras = []
    for frase in frases:
        chunks = [palabra[0] for palabra in frase[:-1]]
        palabras.append("".join(chunks))
    return palabras

def reporte(prediction_tags, real_tags, example):
    print("Letra | Predicción | Real | Es correcto?")
    for prediction, real, letter in zip(prediction_tags, real_tags, extractTokens(example)):
        print(f"{letter} | {prediction} | {real} | {True if prediction == real else False}")
        
def accuracy_score(y_test, y_pred):
    right, total = 0, 0
    for real, prediction in zip(y_test, y_pred):
        if real == prediction:
            right += 1
    return right / len(y_test)

In [5]:
def feature_functions_maker(sent):
    ''' Reglas que configuran las feature functions para entrenamiento

    :param sent: Data as `[[[[[letter, POS, BIO-label],...],words],sents]]`
    :type: list
    :return: list of words with characters as features list:
        [[[[[letterfeatures],POS,BIO-label],letters],words]]
    :rtype: list
    '''

    featurelist = []
    senlen = len(sent)
    # each word in a sentence
    for i in range(senlen):
        word = sent[i]
        wordlen = len(word)
        lettersequence = ''
        # each letter in a word
        for j in range(wordlen):
            letter = word[j][0]
            # gathering previous letters
            lettersequence += letter
            # ignore digits
            if not letter.isdigit():
                features = [
                    'bias',
                    'letterLowercase=' + letter.lower(),
                ]
                
                if j >= 1:
                    features.append('prevletter=' + lettersequence[j-1:j].lower() + '>')
             
            # Add encoding for pysrfsuite
            featurelist.append([f.encode('utf-8') for f in features])
    return featurelist

# Los Peores ejemplos

In [6]:
peores = []
prediction_tags = []
for i, example in enumerate(letter_corpus):
    features = feature_functions_maker(example)
    try:
        prediction_tags = tagger.tag(features)
    except UnicodeDecodeError as e:
        # TODO: Revisar que hacer
        print(f"Error generando {e.object} > {e.object.decode}")
        continue     
    real_tags = extractLabels(example, 1)
    accuracy = accuracy_score(real_tags, prediction_tags)
    if accuracy <= 0.8:
        peores.append((i, accuracy))
peores = sorted(peores, key=lambda t: t[1])
print(len(peores), len(letter_corpus))

420 1786


In [7]:
top = peores[:5]
for i, data in enumerate(top):
    print("-"*50)
    index = data[0]
    palabras = obtener_palabras(corpus[index])
    example = letter_corpus[index]
    features = feature_functions_maker(example)
    try:
        prediction_tags = tagger.tag(features)
    except UnicodeDecodeError as e:
        print(e.object)
        continue
    real_tags = extractLabels(example, 1)
    print(f"Ejemplo {i+1} de 5 | Frase: \"{' '.join(palabras)}\" | Precisión: {data[1]}")
    reporte(prediction_tags, real_tags, example)

--------------------------------------------------
Ejemplo 1 de 5 | Frase: "nuní" | Precisión: 0.0
Letra | Predicción | Real | Es correcto?
n | B-stem | B-det.dem | False
u | I-stem | I-det.dem | False
n | I-stem | I-det.dem | False
í | I-stem | I-det.dem | False
--------------------------------------------------
Ejemplo 2 de 5 | Frase: "ibé" | Precisión: 0.0
Letra | Predicción | Real | Es correcto?
i | I-it | B-3.icp | False
b | B-dual.exc | B-stem | False
é | I-dual.exc | I-stem | False
--------------------------------------------------
Ejemplo 3 de 5 | Frase: "xirembik'íhí" | Precisión: 0.08333333333333333
Letra | Predicción | Real | Es correcto?
x | B-stem | B-3.prf | False
i | I-stem | I-3.prf | False
r | I-stem | B-stem | False
e | I-stem | I-stem | True
m | B-med | I-stem | False
b | B-3.cpl | I-stem | False
i | I-3.cpl | I-stem | False
k | B-stem | B-3.obj | False
' | I-stem | I-3.obj | False
í | I-stem | I-3.obj | False
h | I-stem | B-pl | False
í | I-stem | I-pl | False
-----

# Mejores ejemplos

Como de esos hay muchos tomaremos 3 al azar

In [9]:
flag = 0
while flag != 3:
    index = random.randint(0, len(letter_corpus))
    palabras = obtener_palabras(corpus[index])
    example = letter_corpus[index]
    features = feature_functions_maker(example)
    prediction_tags = tagger.tag(features)
    real_tags = extractLabels(example, 1)
    accuracy = accuracy_score(real_tags, prediction_tags)
    if accuracy >= 0.99:
        print(f"Ejemplo {flag + 1} de 3 | Frase: \"{' '.join(palabras)}\" | Precisión: {accuracy}")
        reporte(prediction_tags, real_tags, example)
        flag += 1
        print("-"*50)

Ejemplo 1 de 3 | Frase: "gípíh nι rι bähtsí" | Precisión: 1.0
Letra | Predicción | Real | Es correcto?
g | B-2.icp | B-2.icp | True
í | I-2.icp | I-2.icp | True
p | B-stem | B-stem | True
í | I-stem | I-stem | True
h | B-lig | B-lig | True
n | B-stem | B-stem | True
ι | I-stem | I-stem | True
r | B-det | B-det | True
ι | I-det | I-det | True
b | B-stem | B-stem | True
ä | I-stem | I-stem | True
h | I-stem | I-stem | True
t | I-stem | I-stem | True
s | I-stem | I-stem | True
í | I-stem | I-stem | True
--------------------------------------------------
Ejemplo 2 de 3 | Frase: "dótsíbé k'ι rι hmíyι'wi gehña dómábé" | Precisión: 1.0
Letra | Predicción | Real | Es correcto?
d | B-1.cpl | B-1.cpl | True
ó | I-1.cpl | I-1.cpl | True
t | B-stem | B-stem | True
s | I-stem | I-stem | True
í | I-stem | I-stem | True
b | B-dual.exc | B-dual.exc | True
é | I-dual.exc | I-dual.exc | True
k | B-stem | B-stem | True
' | I-stem | I-stem | True
ι | I-stem | I-stem | True
r | B-det | B-det | True
ι | I-d